# Project 02 - Image Processing


## Student Information


- Full name: Lê Phước Phát
- Student ID: 22127322
- Class: 22CLC10


## Required Libraries


In [2]:
# IMPORT YOUR LIBS HERE
import numpy as np  # import thu vien numpy (tinh toan ma tran)
from PIL import Image  # import thu vien pillow (doc, ghi anh)
import matplotlib.pyplot as plt  # import thu vien matplotlib (hien thi anh)
import os


## Function Definitions


In [40]:
def read_img(img_path):
    """
    Read image from img_path

    Parameters
    ----------
    img_path : str
        Path of image

    Returns
    -------
        Image
    """

    # Check if the image path is not a string.
    if not isinstance(img_path, str):
        raise ValueError("The provided path is not a string !!!")
    # Check if the image path does not exist.
    if not os.path.exists(img_path):
        raise FileNotFoundError("The specified image file does not exist !!!")

    try:
        with Image.open(img_path) as img:
            img_2d = np.array(img)
    except Exception as e:
        raise IOError(f"An error occurred while reading the image: {e}")
    return img_2d


def show_img(img):
    """
    Show image

    Parameters
    ----------
    img : <your type>
        Image
    """

    # Checks if img_2d is not a numpy array
    if not isinstance(img, np.ndarray):
        raise ValueError("Image img_2d should be a numpy array !!!")

    plt.figure(figsize=(8, 8))
    plt.imshow(img)  # showing the image 2d with shape that is (height, width, channels)
    plt.axis("off")  # turn off the axis => make image perfect to show
    plt.show()  # showing the graph that contains the image


def save_img(img, img_path):
    """
    Save image to img_path

    Parameters
    ----------
    img : <your type>
        Image
    img_path : str
        Path of image
    """

    # checks if the image is not a numpy array.
    if not isinstance(img, np.ndarray):
        raise ValueError("Image should be a numpy array !!!")
    # checks if the path of image is not a string.
    if not isinstance(img_path, str):
        raise ValueError("The path of image should be a string !!!")

    if img.ndim == 2:  # Grayscale image
        image = Image.fromarray(np.uint8(img), mode="L")
    elif img.ndim == 3 and img.shape[2] == 3:  # RGB image
        image = Image.fromarray(np.uint8(img), mode="RGB")
    else:
        raise ValueError("Unsupported image format")

    image.save(img_path)


# --------------------------------------------------------------------------------
# 1. Adjust Brightness
def adjust_brightness(image, bias: int):
    """
    Adjust the brightness of the image.

    This function modifies the brightness of an image by adding a specified brightness
    value for each pixel's intensity. The pixel values are clipped to ensure they remain
    within the valid range of 0 to 255.

    Args:
        image (np.ndarray): a numpy array representing the image. It should be
                            in an 8-bit format (values ranging from 0 to 255)
        brightness (int): the amount by which to adjust the brightness.
                            Can be positive values (to increase brightness) or negative values (to decrease brightness).

    Returns:
        np.ndarray: a numpy array of the same shape as the input image, with brightness adjusted. The output image is
        also in an 8-bit format, with pixel values clipped between 0 and 255.
    """
    arr = image.astype(np.int16)  # Convert to init16 to avoid overflow.
    arr = np.clip(arr + bias, 0, 255)  # Adjust brightness and clip values
    return arr.astype(np.uint8)  # Convert back to uint8


# 2. Adjust Contrast
def adjust_contrast(image, bias: float):
    """
    Adjusts the contrast of an image.

    This function modifies the contrast of an image by multiplying each pixel's intensity by a specified scalar value.
    The pixel values are clipped to ensure they remain within the valid range of 0 to 255.

    Args:
        image (np.ndarray): a numpy array representing the image.
                            It should be in an 8-bit format (values ranging from 0 to 255).
                            The array can be grayscale or RGB (3 channels).
        scalar (float): the factor by which to adjust the contrast.
                        Values greater than 1 increase contrast.
                        Values between 0 and 1 decrease contrast.
                        A value of 1 will not change the contrast.
    Returns:
        np.ndarray: a numpy array of the same shape as the input image, with contrast adjusted.
                    The output image is also in an 8-bit format, with pixel values are clipped between 0 and 255.
    """
    arr = image.astype(np.int16)  # Convert to int16 to avoid overflow.
    arr = np.clip(arr * bias, 0, 255)  # Adjust contrast and clip values.
    return arr.astype(np.uint8)  # Convert back to uint8.


# 3. Flip the image (Vertical / Horizontal)
def flip_image(image, mode):
    """
    Flips an image either horizontally or vertically.

    The function flips the input image based on the specified mode.
    A horizontal flip mirrors the image across the vertical axis,
    while a vertical flip mirrors the image across the horizontal axis.

    Args:
        image (np.ndarray): a numpy array representing the image. The array can be
                            grayscale or RGB (3 channels).
        mode (str): The mode of flipping. It can be either "horizontal" to flip the
                    image horizontally or "vertical" to flip the image vertically.

    Returns:
        np.ndarray: a numpy array of the same input image, with the image flipped according to the specified mode.

    Raises:
        ValueError: If the mode is not "horizontal" or "vertical"
    """
    if mode == "horizontal":
        arr = np.flip(image, axis=1)  # flip image horizontally
    elif mode == "vertical":
        arr = np.flip(image, axis=0)  # flip image vertically
    else:
        raise ValueError("Mode must be 'horizontal' or 'vertical'.")
    return arr


# 4. Convert the RGB image into grayscale/sepia
# 4.1 Convert the RGB image into grayscale image
def rgb_to_grayscale(image):
    """
    Converts an RGB image to a grayscale image.

    This function takes an RGB image and converts it to a grayscale image
    by applying the standard luminance formula. The formula uses the
    weighted sum of the R, G, and B components to account for human
    perception of different colors.

    Args:
        image (np.ndarray): a numpy array representing the RGB image.
                            The array should have shape (height, width, 3) with 3 channels (R, G, B)
                            and values ranging from 0 to 255.
    Returns:
        np.ndarray: a numpy array representing the grayscale image, with the same width and height as the input image.
                    The output image is in an 8-bit format and has a single channel with values ranging from 0 to 255.

    References:
        - https://support.ptc.com/help/mathcad/r10.0/en/index.html#page/PTC_Mathcad_Help/example_grayscale_and_color_in_images.html
        - https://www.baeldung.com/cs/convert-rgb-to-grayscale
    """

    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("Input image must be an RGB image with 3 channels.")

    # Step 1: Trích xuất các kênh màu
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel = image[:, :, 2]

    # Step 2: Nhân mỗi kênh với trọng số tương ứng
    red_weighted = 0.299 * red_channel
    green_weighted = 0.587 * green_channel
    blue_weighted = 0.114 * blue_channel

    # Step 3: Cộng các giá trị trọng số để tạo ra giá trị ảnh xám
    gray = red_weighted + green_weighted + blue_weighted

    # Clip values to ensure they are within the valid range of 0 to 255
    gray = np.clip(gray, 0, 255)
    return gray.astype(np.uint8)  # convert to uint8 format


# 4.2 Covert the RGB image into sepia image
def rgb_to_sepia(image):
    """
    Converts an RGB image to a sepia-toned image.

    This function takes an RGB image and applies a sepia tone effect by
    transforming the color values using a specific formula.
    The resulting image has a warm, brownish tone that is characteristic of sepia.

    Args:
        image (np.ndarray): a numpy array representing the RGB image.
                            The array should have 3 channels (R, G, B),
                            and values ranging from 0 to 255.

    Returns:
        np.ndarray: a numpy array representing the sepia-toned image, with the same shape as the input image.
                    The output array has 3 channels (R, G, B) with values ranging from 0 to 255.

    References:
        - https://dyclassroom.com/image-processing-project/how-to-convert-a-color-image-into-sepia-image
    """
    red = image[:, :, 0]
    green = image[:, :, 1]
    blue = image[:, :, 2]

    tr = (
        0.393 * red + 0.769 * green + 0.189 * blue
    )  # calculate sepia tone for red channel
    tg = (
        0.349 * red + 0.686 * green + 0.168 * blue
    )  # calculate sepia tone for green channel
    tb = (
        0.272 * red + 0.534 * green + 0.131 * blue
    )  # calculate sepia tone for blue channel
    sepia = np.stack([tr, tg, tb], axis=2)  # Stack channels to form sepia image
    sepia = np.clip(sepia, 0, 255)  # Clip values to ensure they are within [0, 255]
    return sepia.astype(np.uint8)  # Convert to uint8 format


# 5. Blur the image and sharpen the image
def apply_kernel(image, kernel):
    """
    Applies a convolutional kernel to an image.

    This function takes an image and a convolutional kernel, and applies the kernel to the image using convolution.
    This image is padded to ensure the output image has the same dimensions as the input image.

    Args:
        image (np.ndarray): a numpy array representing the image.
                            The array can be grayscale or RGB (3 channels).
        kernel (np.ndarray): a 2D numpy array representing the convolutional kernel to be applied to the image.

    Returns:
        np.ndarray: a numpy array of the same shape as the input image, with the kernel applied.
                    The output array has the same number of channels as the input image.
    """
    k_height, k_width = (
        kernel.shape
    )  # Get the dimensions of the kernel with k_height is the height of the kernel and k_width is the width of the kernel
    img_height, img_width = image.shape[
        :2
    ]  # Get the dimensions of the image with img_height is the height of the image and img_width is the width of the image.
    pad_h, pad_w = (
        k_height // 2,
        k_width // 2,
    )  # Calculate padding size for height and width.
    padded_image = np.pad(
        image, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode="constant"
    )  # Pad the image with zeros on all sides
    new_image = np.zeros_like(image)  # Initialize an array to store the new image
    for i in range(img_height):  # Loop through each pixel in the image (height)
        for j in range(img_width):  # Loop through each pixel in the image (width)
            for k in range(3):  # Loop through each channel (R, G, B)
                new_image[i, j, k] = np.sum(
                    kernel * padded_image[i : i + k_height, j : j + k_width, k]
                )  # Apply the kernel to the current window of the image
    new_image = np.clip(
        new_image, 0, 255
    )  # Clip the values to be in the range [0, 255]
    return new_image.astype(np.uint8)  # Convert the result to uint8 format and return.


# 5.1. Blur the image
def blur_image(image):
    """
    Applies a blurring effect to an image.

    This function blurs the input image by applying a convolutional kernel that averages
    the pixel values in a 5x5 neighborhood.

    Args:
        image (np.ndarray): a numpy array representing the image. The array can be grayscale or RGB (3 channels).

    Returns:
        np.ndarray: a numpy array of the same shape as the input image with a blurring effect applied.
    """
    kernel = (
        np.ones((5, 5)) / 25
    )  # Define the blur kernel as a 5x5 matrix with all elements equal to 1/25
    return apply_kernel(
        image, kernel
    )  # Apply the kernel to the image using the apply_kernel function


# 5.2. Sharpen the image
def sharpen_image(image):
    """
    Applies a sharpening effect to an image.

    This function sharpens the input image by applying a convolutional kernel that enhances the edges in the image.

    Args:
        image (np.ndarray): a numpy array representing the image.
                            The array can be grayscale or RGB (3 channels).

    Returns:
        np.ndarray: a numpy array of the same shape as the input image, with a sharpening effect applied.
    """
    kernel = np.array(
        [[0, -1, 0], [-1, 5, -1], [0, -1, 0]]
    )  # Define the sharpen kernel as a 3x3 matrix.
    return apply_kernel(
        image, kernel
    )  # Apply the kernel to the image using the apply_kernel function


# 6. Crop the image to size (crop in center)
def crop_center(image, new_width, new_height):
    """
    Crops the image to a specified size by taking the center portion of the image.

    Args:
        image (np.ndarray): a numpy array representing the image. The array can be grayscale or RGB (3 channels).
        new_width (int): the width of the cropped image.
        new_height (int): the height of the cropped image.

    Returns:
        np.ndarray: a numpy array of the same shape as the input image, cropped to the specified size.
    """
    # Get the dimensions of the image
    height, width = image.shape[:2]

    # Calculate the left, top, right, and bottom coordinates for cropping
    left = (width - new_width) // 2
    top = (height - new_height) // 2
    right = (width + new_width) // 2
    bottom = (height + new_height) // 2

    # Crop the image using the calculated coordinates
    return image[top:bottom, left:right]


# 7. Crop the image by frame
# 7.1. Circle frame
def crop_circle(image):
    # Bước 1: Lấy kích thước của hình ảnh
    height, width = image.shape[:2]

    # Bước 2: Tạo lưới tọa độ cho hình ảnh
    y, x = np.ogrid[:height, :width]

    # Bước 3: Tính toán tọa độ của tâm hình ảnh
    center_y, center_x = height // 2, width // 2

    # Bước 4: Tính toán bán kính của hình tròn
    radius = min(center_y, center_x)

    # Bước 5: Tạo mặt nạ cho hình tròn
    mask = (y - center_y) ** 2 + (x - center_x) ** 2 <= radius**2

    # Bước 6: Áp dụng mặt nạ cho hình ảnh
    result = np.zeros_like(
        image
    )  # Tạo một hình ảnh đen cùng kích thước với hình ảnh gốc
    result[mask] = image[mask]  # Giữ lại các pixel nằm trong hình tròn

    return result


# 7.2. Elliptical cross frame
def crop_elliptical_cross(img):
    """
    Crop the image with a two ellipses symmetrically overlapping

    Input:
        img: np.ndarray
        An image with in ndarry format

    Output:
        ellipses_img: np.ndarray
        A new image that have been cropped with a the ellipses mask
    """
    # Get the width and height of the image and create a copy of the original image to process on
    height, width = image.shape[:2]
    ellipses_img = img.copy()

    # Calculate the center, the semi-major, semi-minor of the ellipse and the radius
    center_x = width // 2
    center_y = height // 2
    s_major = width
    s_minor = width / (np.sqrt(2) + 1)
    radius = min(center_y, center_x)

    # Creates two 2D arrays Y and X representing the grid of pixel
    y, x = np.ogrid[:height, :width]

    # Caluculate the arguments for calculation
    dist1 = (x - center_y) + (y - center_x)
    dist2 = (x - center_y) - (y - center_x)

    # Calculate the Euclidean distance from each pixel to the center of the image
    # Creates a boolean mask where the value is True for pixels inside circular region
    mask1 = (-dist2) ** 2 / (s_major * np.sqrt(2)) + (dist1) ** 2 / (
        s_minor * np.sqrt(2)
    ) <= radius
    mask2 = (dist1) ** 2 / (s_major * np.sqrt(2)) + (-dist2) ** 2 / (
        s_minor * np.sqrt(2)
    ) <= radius

    # Sets the pixel values outside the ellipses mask to 0
    ellipses_img[~(mask1 | mask2)] = 0

    return ellipses_img


# 8. Zoom in / zoom out the image
# 8.1. Zoom in the image
def zoom_in(image, factor=2):
    """
    Zooms in on the image by a given factor.

    This function increases the size of the image by the specified zoom factor. The zoom is performed by
    resizing the image and interpolating pixel values.

    Parameters
    ----------
    image : np.ndarray
        The input image array. Can be grayscale or RGB (3 channels).
    factor : float
        The zoom-in factor. The image will be enlarged by this factor.

    Returns
    -------
    np.ndarray
        The zoomed-in image array. The size of the image will be increased by the zoom factor.
    """
    # Get the dimensions of the original image
    height, width = image.shape[:2]

    # Calculate the new dimensions based on the zoom factor
    new_height, new_width = int(height * factor), int(width * factor)

    # Initialize the zoomed-in image with zeros
    zoomed_image = np.zeros((new_height, new_width, image.shape[2]), dtype=image.dtype)

    # Populate the zoomed-in image with pixel values from the original image
    for i in range(new_height):
        for j in range(new_width):
            # Map the coordinates in the zoomed image to the original image
            zoomed_image[i, j] = image[int(i / factor), int(j / factor)]

    return zoomed_image


# 8.2. Zoom out the image
def zoom_out(image, factor=2):
    """
    Zooms out of the image by a given factor.

    This function decreases the size of the image by the specified zoom factor. The zoom is performed by
    resizing the image and interpolating pixel values.

    Parameters
    ----------
    image : np.ndarray
        The input image array. Can be grayscale or RGB (3 channels).
    factor : float
        The zoom-out factor. The image will be reduced to this fraction of its original size.

    Returns
    -------
    np.ndarray
        The zoomed-out image array. The size of the image will be reduced by the zoom factor.
    """
    # Get the dimensions of the original image
    height, width = image.shape[:2]

    # Calculate the new dimensions based on the zoom factor
    new_height, new_width = int(height / factor), int(width / factor)

    # Initialize the zoomed-out image with zeros
    zoomed_image = np.zeros((new_height, new_width, image.shape[2]), dtype=image.dtype)

    # Populate the zoomed-out image with pixel values from the original image
    for i in range(new_height):
        for j in range(new_width):
            # Map the coordinates in the zoomed-out image to the original image
            zoomed_image[i, j] = image[int(i * factor), int(j * factor)]

    return zoomed_image


<ins>Note:</ins> For clarity, include docstrings with each function.


## Your tests


## Main FUNCTION


In [41]:
def main():
    while True:
        file_name = input("Enter the image file name (or type 'exit' to quit): ")
        if file_name.lower() == "exit":
            print("Exiting program.")
            break

        try:
            image = read_img(file_name)
        except FileNotFoundError:
            print("File not found. Please enter a valid file name.")
            continue
        except Exception as e:
            print(f"Error reading image: {e}")
            continue

        while True:
            print("\nSelect image processing function:")
            print("1. Adjust brightness")
            print("2. Adjust contrast")
            print("3. Flip image (horizontal/vertical)")
            print("4. Convert RGB image to grayscale/sepia")
            print("5. Blur/sharpen image")
            print("6. Crop image by size (center crop)")
            print("7. Crop image by frame")
            print("8. Zoom in / Zoom out the image")
            print("0. Apply all functions")
            print("9. Change image file")
            print("10. Exit")

            choice = int(input("Your choice: "))

            if choice == 10:
                print("Exiting program.")
                return
            elif choice == 9:
                break
            elif choice == 1 or choice == 0:
                scalar = int(input("Enter scalar value for brightness: "))
                bright_image = adjust_brightness(image, scalar)
                save_img(bright_image, f"{file_name.split('.')[0]}_brightness.png")
                if choice == 1:
                    show_img(bright_image)

            if choice == 2 or choice == 0:
                scalar = float(input("Enter scalar value for contrast: "))
                contrast_image = adjust_contrast(image, scalar)
                save_img(contrast_image, f"{file_name.split('.')[0]}_contrast.png")
                if choice == 2:
                    show_img(contrast_image)

            if choice == 3 or choice == 0:
                print("Select flip mode:")
                print("1. Horizontal")
                print("2. Vertical")
                mode_choice = int(input("Your choice: "))
                mode = "horizontal" if mode_choice == 1 else "vertical"
                flipped_image = flip_image(image, mode)
                save_img(flipped_image, f"{file_name.split('.')[0]}_flipped.png")
                if choice == 3:
                    show_img(flipped_image)

            if choice == 4 or choice == 0:
                gray_image = rgb_to_grayscale(image)
                save_img(gray_image, f"{file_name.split('.')[0]}_grayscale.png")
                sepia_image = rgb_to_sepia(image)
                save_img(sepia_image, f"{file_name.split('.')[0]}_sepia.png")
                if choice == 4:
                    show_img(gray_image)
                    show_img(sepia_image)

            if choice == 5 or choice == 0:
                blurred_image = blur_image(image)
                save_img(blurred_image, f"{file_name.split('.')[0]}_blur.png")
                sharpened_image = sharpen_image(image)
                save_img(sharpened_image, f"{file_name.split('.')[0]}_sharpen.png")
                if choice == 5:
                    show_img(blurred_image)
                    show_img(sharpened_image)

            if choice == 6 or choice == 0:
                new_width = int(input("Enter new width: "))
                new_height = int(input("Enter new height: "))
                cropped_image = crop_center(image, new_width, new_height)
                save_img(cropped_image, f"{file_name.split('.')[0]}_crop_center.png")
                if choice == 6:
                    show_img(cropped_image)

            if choice == 7 or choice == 0:
                print("Select frame type:")
                print("1. Circle")
                print("2. Elliptical cross")
                frame_choice = int(input("Your choice: "))

                if frame_choice == 1:
                    cropped_circle_image = crop_circle(image)
                    save_img(
                        cropped_circle_image,
                        f"{file_name.split('.')[0]}_crop_circle.png",
                    )
                    if choice == 7:
                        show_img(cropped_circle_image)
                elif frame_choice == 2:
                    cropped_ellipse_image = crop_elliptical_cross(image)
                    save_img(
                        cropped_ellipse_image,
                        f"{file_name.split('.')[0]}_crop_elliptical_cross.png",
                    )
                    if choice == 7:
                        show_img(cropped_ellipse_image)

            if choice == 8 or choice == 0:
                try:
                    print("Select zoom option:")
                    print("1. Zoom in")
                    print("2. Zoom out")
                    zoom_choice = int(input("Your choice: "))

                    if zoom_choice == 1:
                        zoomed_image = zoom_in(image, factor=2)
                        save_img(zoomed_image, f"{file_name.split('.')[0]}_zoom_in.png")
                        if choice == 8:
                            show_img(zoomed_image)
                    elif zoom_choice == 2:
                        zoomed_image = zoom_out(image, factor=2)
                        save_img(
                            zoomed_image, f"{file_name.split('.')[0]}_zoom_out.png"
                        )
                        if choice == 8:
                            show_img(zoomed_image)
                    else:
                        print(
                            "Invalid choice. Please select 1 for zoom in or 2 for zoom out."
                        )
                except Exception as e:
                    print(f"Error processing zoom: {e}")
            if choice == 0:
                print("All functions completed.")
                break


In [ ]:
# Call main function
if __name__ == "__main__":
    main()
